In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_for_train.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)
        
        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=te_x[test_i],
                y_eval=te_y[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class KimConvolutionalModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1408.5882.pdf
    '''
    def __init__(self,
        embeddings_configuration,
        conv_configurations = [(3, 100), (4, 100), (5, 100)],
        drop_rate           = 0.5):
        '''Constructor.
        # Parameters:
        embeddings: List of embeddings configuration. Each configuration is a
            pair of the form (embedding, trainable). `embedding` is a numpy
            array and `trainable` is a boolean that indicates whether that
            embedding is trainable or not.
        conv_configurations: List of pairs. Each pair represents a
            convolution configuration. Each configuration determines the
            size and number of each filter.
        '''

        self._embeddings_configuration = embeddings_configuration
        self._conv_configurations = conv_configurations
        self._drop_rate = drop_rate

    def __call__(self, input):
        self._embeddings_tf = tf.stack(
            values = [
                self._create_embedding_layer(e, input)
                for e in self._embeddings_configuration],
            axis = 1
        )

        self._convolutions_tf = self._create_convolutional_layers(
            self._conv_configurations, self._embeddings_tf)
        
        self._add_tf = self._create_add_layers(self._convolutions_tf)

        self._poolings_tf = self._create_maxpooling_layer(
            self._add_tf)

        self._reshape_tf = self._create_reshape_layer(self._poolings_tf)
        self._dropout_tf = tf.nn.dropout(
            self._reshape_tf,
            keep_prob = self._drop_rate)

        return self._dropout_tf

    def summary(self):
        print('embedding:', str(self._embeddings_tf.shape))
        for c in self._convolutions_tf:
            print('conv:', str(c.shape))
        for a in self._add_tf:
            print('add:', str(a.shape))
        for p in self._poolings_tf:
            print('pool:', str(p.shape))
        print('reshape:', str(self._reshape_tf.shape))

    def _create_embedding_layer(self, embedding_configuration, input_x):
        return tf.nn.embedding_lookup(
            params = tf.Variable(
                initial_value = embedding_configuration[0],
                trainable     = embedding_configuration[1]),
            ids = tf.cast(input_x, 'int32')
        )

    def _create_convolutional_layers(self, configuration, input_embedding):
        '''Creates the convolutional layers.
        # Parameters:
        configuration: A list. It must be of the form
            [(filter_size, num_filters), ...]
        # Returns:
        A list of tensorflow nodes. Each node 'i' computes the configuration 'i'.
        '''
        convolutions = []
        for filter_height, num_filters in configuration:
            filter_width = input_embedding.shape[3].value
            filter_shape = [1, filter_height, filter_width, num_filters]

            # Create weights and bias
            W = tf.Variable(
                initial_value=tf.truncated_normal(
                    shape=filter_shape,
                    stddev=0.1))
            b = tf.Variable(
                initial_value=tf.truncated_normal(
                    shape=[num_filters]))

            conv = tf.nn.conv2d(
                input=input_embedding,
                filter=W,
                strides=[1, 1, 1, 1],
                padding="VALID")
            bias = tf.nn.bias_add(conv, b)
            h = tf.nn.relu(bias)
            convolutions.append(h)

        return convolutions

    def _create_add_layers(self, convolutions):
        return [
            tf.reduce_sum(
                input_tensor = c,
                axis=1,
                keepdims=True)
            for c in convolutions
        ]

    def _create_maxpooling_layer(self, tensors):
        '''Creates the maxpooling layer. Computes maxpooling on each node
        # Parameters:
        input_convolutions: List of tensorflow nodes.
        # Returns:
        A list of tensorflow nodes. Each node 'i' computes the maxpooling of node 'i'
        '''
        return [
            tf.reshape(
                tensor = tf.nn.max_pool(
                    value=t,
                    ksize=[1, 1, t.shape[2], 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID'),
                shape = [-1, t.shape[3]]
            )
            for t in tensors
        ]

    def _create_reshape_layer(self, tensors):
        '''Creates a flatten layer
        '''
        return tf.concat(tensors, axis=1)

In [8]:
word_vector = embedding.astype('float32')
model = KimConvolutionalModel(
        embeddings_configuration=[
            (word_vector, True)
        ]
    )

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 1, 41, 100)
conv: (?, 1, 39, 100)
conv: (?, 1, 38, 100)
conv: (?, 1, 37, 100)
add: (?, 1, 39, 100)
add: (?, 1, 38, 100)
add: (?, 1, 37, 100)
pool: (?, 100)
pool: (?, 100)
pool: (?, 100)
reshape: (?, 300)
output: (?, 10)


In [9]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data, test_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, test_data, split_type))

random
Fold:  1
Epoch: 1, Train Loss:  1.871, Train Acc: 43.54%, Val Loss:  1.087, Val Acc: 65.24%, Time: 4.52s *
Epoch: 2, Train Loss: 0.8861, Train Acc: 72.64%, Val Loss: 0.7893, Val Acc: 76.42%, Time: 3.45s *
Epoch: 3, Train Loss: 0.5938, Train Acc: 81.71%, Val Loss: 0.6741, Val Acc: 80.13%, Time: 3.48s *
Epoch: 4, Train Loss: 0.4317, Train Acc: 86.71%, Val Loss: 0.5981, Val Acc: 81.13%, Time: 3.47s *
Epoch: 5, Train Loss: 0.3218, Train Acc: 90.59%, Val Loss: 0.5488, Val Acc: 83.62%, Time: 3.46s *
Epoch: 6, Train Loss: 0.2442, Train Acc: 92.57%, Val Loss: 0.5048, Val Acc: 84.33%, Time: 3.47s *
Epoch: 7, Train Loss: 0.1855, Train Acc: 94.61%, Val Loss: 0.5055, Val Acc: 85.83%, Time: 3.46s *
Epoch: 8, Train Loss: 0.1601, Train Acc: 94.93%, Val Loss: 0.4745, Val Acc: 86.25%, Time: 3.46s *
Epoch: 9, Train Loss: 0.1248, Train Acc: 96.43%, Val Loss:  0.449, Val Acc: 87.39%, Time: 3.46s *
Epoch: 10, Train Loss: 0.1033, Train Acc: 97.07%, Val Loss: 0.4918, Val Acc: 86.61%, Time: 3.61s 
Epoc

Epoch: 19, Train Loss: 0.03944, Train Acc: 98.78%, Val Loss: 0.5383, Val Acc: 87.68%, Time: 3.46s 
Epoch: 20, Train Loss: 0.0376, Train Acc: 98.80%, Val Loss: 0.5114, Val Acc: 88.60%, Time: 3.45s 
Epoch: 21, Train Loss: 0.04156, Train Acc: 98.81%, Val Loss: 0.5238, Val Acc: 87.82%, Time: 3.46s 
Epoch: 22, Train Loss: 0.0344, Train Acc: 98.96%, Val Loss: 0.5159, Val Acc: 88.03%, Time: 3.47s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.88      0.85      0.87       123
                   Filter       0.83      0.85      0.84       167
    Compute Derived Value       0.85      0.89      0.87       159
            Find Extremum       0.90      0.90      0.90       167
                     Sort       0.97      0.92      0.94       125
          Determine Range       0.79      0.87      0.83       128
Characterize Distribution       0.90      0.89      

Epoch: 29, Train Loss: 0.0268, Train Acc: 99.24%, Val Loss: 0.5961, Val Acc: 87.89%, Time: 3.46s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.86      0.90      0.88       150
                   Filter       0.87      0.81      0.84       137
    Compute Derived Value       0.87      0.87      0.87       159
            Find Extremum       0.85      0.95      0.90       160
                     Sort       0.96      0.92      0.94       124
          Determine Range       0.90      0.83      0.86       129
Characterize Distribution       0.91      0.90      0.91       124
           Find Anomalies       0.88      0.92      0.90       145
                  Cluster       0.90      0.93      0.91       121
                Correlate       0.94      0.87      0.90       155

                micro avg       0.89      0.89      0.89      1404
            

Epoch: 1, Train Loss:  1.901, Train Acc: 42.87%, Val Loss:  1.127, Val Acc: 66.00%, Time: 3.62s *
Epoch: 2, Train Loss: 0.9008, Train Acc: 71.76%, Val Loss: 0.8487, Val Acc: 73.13%, Time: 3.47s *
Epoch: 3, Train Loss: 0.6034, Train Acc: 81.41%, Val Loss: 0.6835, Val Acc: 77.76%, Time: 3.46s *
Epoch: 4, Train Loss: 0.4387, Train Acc: 86.62%, Val Loss: 0.6137, Val Acc: 80.90%, Time: 3.48s *
Epoch: 5, Train Loss: 0.3249, Train Acc: 90.09%, Val Loss: 0.5869, Val Acc: 82.61%, Time: 3.45s *
Epoch: 6, Train Loss: 0.2556, Train Acc: 92.27%, Val Loss: 0.5312, Val Acc: 83.54%, Time: 3.45s *
Epoch: 7, Train Loss: 0.1902, Train Acc: 94.23%, Val Loss: 0.5137, Val Acc: 83.68%, Time: 3.46s *
Epoch: 8, Train Loss: 0.1535, Train Acc: 95.42%, Val Loss: 0.4576, Val Acc: 86.32%, Time: 3.47s *
Epoch: 9, Train Loss: 0.1251, Train Acc: 96.17%, Val Loss: 0.4797, Val Acc: 86.74%, Time: 3.46s *
Epoch: 10, Train Loss: 0.1084, Train Acc: 96.79%, Val Loss: 0.5064, Val Acc: 85.74%, Time: 3.44s 
Epoch: 11, Train Los

expert
Fold:  1
Epoch: 1, Train Loss:  1.823, Train Acc: 43.26%, Val Loss:  1.722, Val Acc: 44.24%, Time: 3.69s *
Epoch: 2, Train Loss: 0.8855, Train Acc: 72.49%, Val Loss:  1.814, Val Acc: 48.78%, Time: 3.60s *
Epoch: 3, Train Loss: 0.6116, Train Acc: 80.96%, Val Loss:  1.903, Val Acc: 48.78%, Time: 3.51s 
Epoch: 4, Train Loss: 0.4409, Train Acc: 86.85%, Val Loss:  2.055, Val Acc: 48.62%, Time: 3.50s 
Epoch: 5, Train Loss: 0.3268, Train Acc: 89.85%, Val Loss:  2.156, Val Acc: 50.81%, Time: 3.49s *
Epoch: 6, Train Loss:  0.248, Train Acc: 92.34%, Val Loss:  2.279, Val Acc: 52.19%, Time: 3.52s *
Epoch: 7, Train Loss: 0.1952, Train Acc: 94.40%, Val Loss:  2.558, Val Acc: 51.62%, Time: 3.50s 
Epoch: 8, Train Loss: 0.1526, Train Acc: 95.44%, Val Loss:  2.686, Val Acc: 50.57%, Time: 3.50s 
Epoch: 9, Train Loss: 0.1308, Train Acc: 96.02%, Val Loss:  2.873, Val Acc: 52.11%, Time: 3.54s 
Epoch: 10, Train Loss: 0.1033, Train Acc: 97.05%, Val Loss:  2.984, Val Acc: 51.87%, Time: 3.50s 
No optimi

...

Epoch: 1, Train Loss:  2.582, Train Acc: 29.88%, Val Loss:  1.528, Val Acc: 49.14%, Time: 3.65s *
Epoch: 2, Train Loss:  1.249, Train Acc: 60.06%, Val Loss:  1.244, Val Acc: 62.52%, Time: 3.46s *
Epoch: 3, Train Loss: 0.9201, Train Acc: 70.88%, Val Loss:   1.18, Val Acc: 66.38%, Time: 3.54s *
Epoch: 4, Train Loss: 0.6983, Train Acc: 78.45%, Val Loss:  1.162, Val Acc: 67.31%, Time: 3.45s *
Epoch: 5, Train Loss: 0.5423, Train Acc: 83.28%, Val Loss:  1.151, Val Acc: 68.88%, Time: 3.44s *
Epoch: 6, Train Loss:  0.425, Train Acc: 86.87%, Val Loss:  1.182, Val Acc: 69.96%, Time: 3.46s *
Epoch: 7, Train Loss: 0.3415, Train Acc: 89.60%, Val Loss:  1.149, Val Acc: 71.24%, Time: 3.44s *
Epoch: 8, Train Loss: 0.2704, Train Acc: 92.25%, Val Loss:  1.244, Val Acc: 70.74%, Time: 3.46s 
Epoch: 9, Train Loss: 0.2201, Train Acc: 93.69%, Val Loss:  1.281, Val Acc: 71.10%, Time: 3.47s 
Epoch: 10, Train Loss: 0.1786, Train Acc: 94.85%, Val Loss:  1.302, Val Acc: 70.60%, Time: 3.44s 
Epoch: 11, Train Loss:

Epoch: 6, Train Loss: 0.4482, Train Acc: 86.34%, Val Loss:  1.078, Val Acc: 68.41%, Time: 3.46s *
Epoch: 7, Train Loss: 0.3575, Train Acc: 89.30%, Val Loss:   1.09, Val Acc: 68.98%, Time: 3.45s *
Epoch: 8, Train Loss: 0.2854, Train Acc: 91.48%, Val Loss:  1.151, Val Acc: 67.35%, Time: 3.44s 
Epoch: 9, Train Loss: 0.2263, Train Acc: 93.27%, Val Loss:  1.205, Val Acc: 68.06%, Time: 3.45s 
Epoch: 10, Train Loss: 0.1905, Train Acc: 94.37%, Val Loss:   1.29, Val Acc: 68.98%, Time: 3.47s *
Epoch: 11, Train Loss: 0.1499, Train Acc: 95.64%, Val Loss:  1.297, Val Acc: 68.48%, Time: 3.44s 
Epoch: 12, Train Loss:  0.127, Train Acc: 96.40%, Val Loss:  1.367, Val Acc: 68.76%, Time: 3.44s 
Epoch: 13, Train Loss: 0.1059, Train Acc: 96.82%, Val Loss:  1.377, Val Acc: 69.47%, Time: 3.44s *
Epoch: 14, Train Loss: 0.09098, Train Acc: 97.29%, Val Loss:  1.479, Val Acc: 69.61%, Time: 3.44s *
Epoch: 15, Train Loss: 0.08245, Train Acc: 97.59%, Val Loss:  1.473, Val Acc: 70.81%, Time: 3.45s *
Epoch: 16, Train

Epoch: 1, Train Loss:  2.056, Train Acc: 38.66%, Val Loss:  1.395, Val Acc: 53.94%, Time: 3.72s *
Epoch: 2, Train Loss:  1.019, Train Acc: 67.56%, Val Loss:  1.093, Val Acc: 65.06%, Time: 3.59s *
Epoch: 3, Train Loss: 0.7048, Train Acc: 78.09%, Val Loss: 0.9819, Val Acc: 69.39%, Time: 3.67s *
Epoch: 4, Train Loss: 0.5128, Train Acc: 84.43%, Val Loss: 0.9533, Val Acc: 70.28%, Time: 3.56s *
Epoch: 5, Train Loss: 0.3875, Train Acc: 88.28%, Val Loss:  1.068, Val Acc: 69.78%, Time: 3.57s 
Epoch: 6, Train Loss: 0.2933, Train Acc: 90.92%, Val Loss:  1.071, Val Acc: 70.96%, Time: 3.55s *
Epoch: 7, Train Loss: 0.2333, Train Acc: 92.99%, Val Loss:  1.185, Val Acc: 70.08%, Time: 3.56s 
Epoch: 8, Train Loss: 0.1873, Train Acc: 94.48%, Val Loss:  1.128, Val Acc: 72.24%, Time: 3.53s *
Epoch: 9, Train Loss:  0.141, Train Acc: 95.85%, Val Loss:  1.192, Val Acc: 72.93%, Time: 3.54s *
Epoch: 10, Train Loss: 0.1257, Train Acc: 96.04%, Val Loss:  1.224, Val Acc: 71.56%, Time: 3.55s 
Epoch: 11, Train Loss:

Epoch: 3, Train Loss: 0.7015, Train Acc: 78.37%, Val Loss:  1.287, Val Acc: 59.57%, Time: 3.33s *
Epoch: 4, Train Loss: 0.5177, Train Acc: 84.06%, Val Loss:  1.301, Val Acc: 59.30%, Time: 3.30s 
Epoch: 5, Train Loss: 0.3985, Train Acc: 88.00%, Val Loss:  1.302, Val Acc: 62.73%, Time: 3.34s *
Epoch: 6, Train Loss:  0.302, Train Acc: 90.82%, Val Loss:  1.335, Val Acc: 64.15%, Time: 3.33s *
Epoch: 7, Train Loss: 0.2301, Train Acc: 93.20%, Val Loss:  1.375, Val Acc: 63.57%, Time: 3.31s 
Epoch: 8, Train Loss: 0.1775, Train Acc: 95.05%, Val Loss:  1.438, Val Acc: 66.26%, Time: 3.32s *
Epoch: 9, Train Loss: 0.1463, Train Acc: 95.81%, Val Loss:  1.495, Val Acc: 65.84%, Time: 3.31s 
Epoch: 10, Train Loss: 0.1262, Train Acc: 96.04%, Val Loss:  1.573, Val Acc: 65.26%, Time: 3.32s 
Epoch: 11, Train Loss: 0.1049, Train Acc: 96.75%, Val Loss:  1.572, Val Acc: 65.26%, Time: 3.31s 
Epoch: 12, Train Loss: 0.08295, Train Acc: 97.54%, Val Loss:  1.758, Val Acc: 65.31%, Time: 3.38s 
No optimization for a 

Epoch: 6, Train Loss: 0.2717, Train Acc: 91.89%, Val Loss:  1.858, Val Acc: 63.64%, Time: 3.70s *
Epoch: 7, Train Loss: 0.2208, Train Acc: 93.52%, Val Loss:  2.007, Val Acc: 60.68%, Time: 3.74s 
Epoch: 8, Train Loss:  0.171, Train Acc: 95.11%, Val Loss:  2.055, Val Acc: 63.21%, Time: 3.70s 
Epoch: 9, Train Loss: 0.1363, Train Acc: 96.08%, Val Loss:  2.089, Val Acc: 60.89%, Time: 3.70s 
Epoch: 10, Train Loss: 0.1142, Train Acc: 96.43%, Val Loss:  2.026, Val Acc: 62.16%, Time: 3.70s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.51      0.60      0.55        30
                   Filter       0.33      0.28      0.30        29
    Compute Derived Value       0.26      0.29      0.27        79
            Find Extremum       0.61      0.57      0.59        74
                     Sort       0.74      0.97      0.84        30
          Determine Range 